# Ejercicios Kata 10
+ Uso de la salida de excepción para depuración.
+ Detección y generación de excepciones.
+ Afectaciones a la lógica de programas cuando se producen excepciones.

## Tracebacks para buscar errores

Traceback = el cuerpo del texto que puede apuntar el origen y final de un error no controlado

Las excepciones son útiles en la toma de decisiones generando mensajes de error descriptivos, ayudando a controlar los problemas operados e inesperados.

In [1]:
# si intentamos abrir un archivo inexistente pasa lo siguiente:
open("/path/to/mars.jpg")

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/mars.jpg'

Lo anterior se puede interpretar de la siguiente manera:
1. El traceback menciona el orden de la salida.
2. Informa el tipo de archivo en la segunda línea de la entrada.
3. El error es FileNotFoundError (el nombre de excepción), lo que significa que el archivo no existe o quizás el directorio correspondiente no existe.

## Try y Except

Si no existe un archivo o directorio, se genera FileNotFoundError, pero si queremos controlar esa excepción, podemos hacerlo con un bloque try y except:

In [4]:
try:
     open('config.txt')
except FileNotFoundError:
     print("Couldn't find the config.txt file!")

Couldn't find the config.txt file!


Después de la palabra clave try
1. Agregamos código que tenga la posibilidad de producir una excepción
2. Agregamos la palabra clave except junto con la posible excepción,
3. seguida de cualquier código que deba ejecutarse cuando se produce esa condición.

Esto evita un seguimiento y sigue informando al usuario sobre el problema.

Aunque es común un archivo que no existe, no es el único error que podemos encontrar. Los permisos de archivo no válidos pueden impedir la lectura de un archivo aunque exista. El archivo tiene código que busca y lee el archivo de configuración del sistema de navegación:

In [5]:
def main():
    try:
        configuration = open('config.txt')
    except FileNotFoundError:
        print("Couldn't find the config.txt file!")


if __name__ == '__main__':
    main()

Couldn't find the config.txt file!


Una manera poco útil de controlar este error sería detectar todas las excepciones posibles para evitar un traceback. Para comprender por qué detectar todas las excepciones es problemático, probaremos actualizando la función main()

In [6]:
def main():
    try:
        configuration = open('config.txt')
    except Exception:
        print("Couldn't find the config.txt file!")

El archivo existe, pero tiene permisos diferentes y Python no puede leerlo. Cuando se trata con errores de software, puede resultar frustrante tener errores que hagan lo siguiente:

+ No indiquen cuál es el problema real.
+ Proporcionen una salida que no coincida con el problema real.
+ No sugieran lo que se puede hacer para corregir el problema.

Vamos a corregir este fragmento de código para abordar todas estas frustraciones. Revertiremos la detección de FileNotFoundError y luego agregamos otro bloque except para detectar PermissionError

In [7]:
def main():
    try:
        configuration = open('config.txt')
    except FileNotFoundError:
        print("Couldn't find the config.txt file!")
    except IsADirectoryError:
        print("Found config.txt but it is a directory, couldn't read it")

Cuando los errores son de una naturaleza similar y no es necesario controlarlos individualmente, puedes agrupar las excepciones como una usando paréntesis en la línea except. Por ejemplo, si el sistema de navegación está bajo cargas pesadas y el sistema de archivos está demasiado ocupado, tiene sentido detectar BlockingIOError y TimeOutError juntos:

+ NOTA: evita agrupar muchas excepciones para proporcionar un mensaje de error generalizado

In [8]:
def main():
    try:
        configuration = open('config.txt')
    except FileNotFoundError:
        print("Couldn't find the config.txt file!")
    except IsADirectoryError:
        print("Found config.txt but it is a directory, couldn't read it")
    except (BlockingIOError, TimeoutError):
        print("Filesystem under heavy load, can't complete reading configuration file")

In [10]:
>>> try:
...     open("mars.jpg")
... except FileNotFoundError as err:
...     print("got a problem trying to read the file:", err)

got a problem trying to read the file: [Errno 2] No such file or directory: 'mars.jpg'


as err significa que err se convierte en una variable con el objeto de excepción como valor. Después, usa este valor para imprimir el mensaje de error asociado a la excepción. 

Otra razón para usar esto es acceder directamente a los atributos del error. 
+ Ej. si detecta una excepción OSError más genérica, que es la excepción primaria de FilenotFoundError y PermissionError, podemos diferenciarlas mediante el atributo .errno

In [12]:
>>> try:
...     open("config.txt")
... except OSError as err:
...     if err.errno == 2:
...         print("Couldn't find the config.txt file!")
...     elif err.errno == 13:
...         print("Found config.txt but couldn't read it")

Couldn't find the config.txt file!


## Excepciones por generar
En situaciones que podrían provocar una condición de error al escribir código, resulta útil generar excepciones que permitan que otro código comprenda cuál es el problema.

Generar excepciones también ayuda a tomar decisiones para otro código. 

+ en función del error, el código puede tomar decisiones inteligentes para resolver, solucionar o ignorar un problema.

Ej.
+ Los astronautas limitan su uso de agua a unos 11 litros al día. Vamos a crear una función que, con base al número de astronautas, pueda calcular la cantidad de agua quedará después de un día o más:

In [13]:
def water_left(astronauts, water_left, days_left):
    daily_usage = astronauts * 11
    total_usage = daily_usage * days_left
    total_water_left = water_left - total_usage
    return f"Total water left after {days_left} days is: {total_water_left} liters"

In [14]:
water_left(5, 100, 2)

'Total water left after 2 days is: -10 liters'

Esto no es útil, ya que una carencia en los litros sería un error y el sistema de navegación podría alertar a los astronautas que no hay suficiente agua para todos en dos días.

Si eres un ingeniero(a) que programa el sistema de navegación, podrías generar una excepción en la función water_left() para alertar de la condición de error.

In [15]:
def water_left(astronauts, water_left, days_left):
    daily_usage = astronauts * 11
    total_usage = daily_usage * days_left
    total_water_left = water_left - total_usage
    if total_water_left < 0:
        raise RuntimeError(f"There is not enough water for {astronauts} astronauts after {days_left} days!")
    return f"Total water left after {days_left} days is: {total_water_left} liters"

In [16]:
water_left(5, 100, 2)

RuntimeError: There is not enough water for 5 astronauts after 2 days!

En el sistema de navegación, el código para señalar la alerta ahora puede usar RuntimeError para generar la alerta

In [17]:
try:
    water_left(5, 100, 2)
except RuntimeError as err:
    alert_navigation_system(err)

NameError: name 'alert_navigation_system' is not defined

In [18]:
water_left('3', '200', None)

TypeError: can't multiply sequence by non-int of type 'NoneType'

El error de TypeError no es muy descriptivo en el contexto de lo que espera la función. Actualizaremos la función para que use TypeError, pero con un mensaje mejor:

In [19]:
def water_left(astronauts, water_left, days_left):
    for argument in [astronauts, water_left, days_left]:
        try:
            # If argument is an int, the following operation will work
            argument / 10
        except TypeError:
            # TypError will be raised only if it isn't the right type 
            # Raise the same exception but with a better error message
            raise TypeError(f"All arguments must be of type int, but received: '{argument}'")
    daily_usage = astronauts * 11
    total_usage = daily_usage * days_left
    total_water_left = water_left - total_usage
    if total_water_left < 0:
        raise RuntimeError(f"There is not enough water for {astronauts} astronauts after {days_left} days!")
    return f"Total water left after {days_left} days is: {total_water_left} liters"

In [20]:
water_left("3", "200", None)

TypeError: All arguments must be of type int, but received: '3'